In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the Adult dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 
                'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
df = pd.read_csv(url, names=column_names, skipinitialspace=True, na_values='?')

# Preprocess the data
df = df.dropna()
df['income'] = df['income'].map({'>50K': 1, '<=50K': 0})
sensitive_attribute = 'sex'
label = 'income'

# Split the data
X = df.drop('income', axis=1)
y = df['income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
!pip install tensorflow
!pip install fairlearn
!pip install inFairness


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.1/236.1 MB 36.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 39.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 31.0 MB/s eta 0:00:00
  Using cached scikit_learn-1.5.1-cp39-cp39-macosx_12_0_arm64.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 33.9 MB/s eta 0:00:00
Using cached scikit_learn-1.5.1-cp39-cp39-macosx_12_0_arm64.whl (11.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
  

In [6]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
le = LabelEncoder()

# Apply LabelEncoder to each categorical column
for column in X_train.select_dtypes(include=['object']).columns:
    X_train[column] = le.fit_transform(X_train[column])


In [7]:
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric

# Assuming `income` is your label and `sensitive_attribute` is defined
train_data = BinaryLabelDataset(df=pd.concat([X_train, y_train], axis=1), 
                                label_names=['income'], 
                                protected_attribute_names=[sensitive_attribute])

# Measure bias
metric = BinaryLabelDatasetMetric(train_data, unprivileged_groups=[{sensitive_attribute: 0}], 
                                  privileged_groups=[{sensitive_attribute: 1}])
print(f"Disparate impact before mitigation: {metric.disparate_impact()}")


Disparate impact before mitigation: 0.3647382591137069
